In [6]:
from Class_tree import DecisionTree
from Class_replace_impute_encode import ReplaceImputeEncode
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, confusion_matrix, classification_report 
from pydotplus import graph_from_dot_data
import graphviz
import pandas as pd
import numpy as np


file_path = '/Users/Pawan/Desktop/MS Folder/4th Sem/STAT 656/Week 5/'
df = pd.read_excel(file_path+"CreditHistory_Clean.xlsx")
print("Authentication Data with %i observations & %i attributes.\n" %df.shape, df.head())

# Replace str values of atrribute 'purpose' with float values
list_1 = [0,1,2,3,4,5,6,7,8,9,10]
df['purpose'].replace(['0','1','2','3','4','5','6','7','8','9','X'],list_1, inplace = True)


attribute_map = {
    'age':[0,(1, 120),[0,0]],
    'amount':[0,(0, 100000),[0,0]],
    'duration':[0,(1,1000),[0,0]],
    'checking':[2,(1, 2, 3, 4),[0,0]],
    'coapp':[2,(1,2,3),[0,0]],
    'depends':[1,(1,2),[0,0]],
    'employed':[2,(1,2,3,4,5),[0,0]],
    'existcr':[2,(1,2,3,4),[0,0]],
    'foreign':[1,(1,2),[0,0]],
    'good_bad':[1,('bad', 'good'),[0,0]],
    'history':[2,(0,1,2,3,4),[0,0]],
    'housing':[2,(1,2,3,),[0,0]],
    'installp':[2,(1,2,3,4),[0,0]],
    'job':[2,(1,2,3,4),[0,0]],
    'marital':[2,(1,2,3,4),[0,0]],
    'other':[2,(1,2,3),[0,0]],
    'property':[2,(1,2,3,4),[0,0]],
    'purpose':[1,(0,1,2,3,4,5,6,7,8,9,10),[0,0]],
    'resident':[2,(1,2,3,4),[0,0]],
    'savings':[2,(1,2,3,4,5),[0,0]],
    'telephon':[1,(1,2),[0,0]] }



encoding = None # Categorical encoding:  Use 'SAS', 'one-hot' or None
scale    = None  # Interval scaling:  Use 'std', 'robust' or None
scaling  = 'No'  # Text description for interval scaling

rie = ReplaceImputeEncode(data_map=attribute_map, nominal_encoding=encoding,
                          interval_scale = scale, display=False)

#features_map = rie.draft_features_map(df)
encoded_df = rie.fit_transform(df)


# First Integer Designates Data Type
# 0=Interval, 1=Binary, 2=Nominal, 3=Other (No Changes, do not include)
#good_bad is the name of the binary target
varlist = ['good_bad']
X = encoded_df.drop(varlist, axis=1)
y = encoded_df[varlist]


X_train, X_validate, y_train, y_validate = train_test_split(X,y,test_size = 0.3, random_state=7)

# 10 fold CV for trees with different 'max_depth' varying from 1 to 10

for m in range(0,10):

    dtc = DecisionTreeClassifier(criterion='gini', max_depth = m+1,min_samples_split=5, min_samples_leaf=5)
    dtc = dtc.fit(X_train,y_train)

    features = X_train.columns.values.tolist()
    classes = ['Good', 'Bad']

    FI = list();
    for i in range(0,20):
        FI.append(dtc.feature_importances_[i])
    len(FI)
    FI_df = pd.DataFrame({'Feature':features,'Importance':FI})
    predictions = dtc.predict(X_validate)
    dtc_10_f1 = cross_val_score(dtc, X_train, y_train, cv=10,scoring='f1')
    dtc_10_accuracy = cross_val_score(dtc, X_train, y_train, cv=10,scoring='accuracy')
    dtc_10_precision = cross_val_score(dtc, X_train, y_train, cv=10,scoring='precision')
    dtc_10_recall = cross_val_score(dtc, X_train, y_train, cv=10,scoring='recall')


    dot_data = export_graphviz(dtc, filled=True, rounded=True,class_names=classes, feature_names = features, out_file=None)
    #write tree to png file 'credit_history_tree'
    graph_png = graph_from_dot_data(dot_data)
    graph_path = '/Users/Pawan/Desktop/MS Folder 😇😇✈️/4th Sem/STAT 656/Week 5/'
    graph_png.write_png(graph_path+'credit_history_tree_'+str(m+1)+'.png')
    
    #Display tree in pdf file 'credit_history_tree.pdf'
    #graph_pdf = graphviz.Source(dot_data)
    #graph_pdf.view('credit_history_tree') #Displays tree
    Matrix = np.empty([10,4])
    for n in range(0,10):
        Matrix[n,0] = dtc_10_precision[n]
        Matrix[n,1] = dtc_10_recall[n]
        Matrix[n,2] = dtc_10_f1[n]
        Matrix[n,3] = dtc_10_accuracy[n]


    
    Index = ['1st Fold','2nd Fold','3rd Fold','4th Fold','5th Fold','6th Fold','7th Fold','8th Fold','9th Fold','10th Fold'];
    Columns = ['Precision', 'Recall/TPR', 'F1_Score', 'Accuracy']
    Matrix = pd.DataFrame(Matrix, index = Index, columns = Columns)
    print('\n\nTable of metrics for 10 Folds CV of model with MAX_TREE_DEPTH = '+str(m+1)+'\n')
    print(Matrix)
    print('\nAvg_Precision = '+ str(Matrix['Precision'].mean()))
    print('Avg_Recall = '+ str(Matrix['Recall/TPR'].mean()))
    print('Avg_F1 = '+ str(Matrix['F1_Score'].mean()))
    print('Avg_Accuracy = '+ str(Matrix['Accuracy'].mean()))


# Best model based on 10-fold CV results

dtc = DecisionTreeClassifier(criterion='gini', max_depth = 8,min_samples_split=5, min_samples_leaf=5)
dtc = dtc.fit(X_train,y_train)

features = X_train.columns.values.tolist()
classes = ['Good', 'Bad']

FI = list();
for i in range(0,20):
    FI.append(dtc.feature_importances_[i])
len(FI)
FI_df = pd.DataFrame({'Feature':features,'Importance':FI})
print('\nDisplaying Feature Importance for the best Model:\n')
print(FI_df)
predictions = dtc.predict(X_validate)

acc = accuracy_score(y_validate, predictions)
pre = precision_score(y_validate, predictions)
tpr = recall_score(y_validate, predictions)
f1 =  f1_score(y_validate,predictions)

Matrix = np.empty([1,4])
Matrix[0,0] = pre
Matrix[0,1] = tpr
Matrix[0,2] = f1
Matrix[0,3] = acc
Index = ['Max_depth_8'];
Columns = ['Precision', 'Recall/TPR', 'F1_Score', 'Accuracy']
Matrix = pd.DataFrame(Matrix, index = Index, columns = Columns)
print("\n\nA table of metrics calculated for best model\n")
print(Matrix)

Authentication Data with 1000 observations & 21 attributes.
    age  amount  duration  checking  history purpose  savings  employed  \
0   67    1169         6         1        4       3        5         5   
1   22    5951        48         2        2       3        1         3   
2   49    2096        12         4        4       6        1         4   
3   45    7882        42         1        2       2        1         4   
4   53    4870        24         1        3       0        1         3   

   installp  marital    ...     resident  property  other  housing  existcr  \
0         4        3    ...            4         1      3        2        2   
1         2        2    ...            2         1      3        2        1   
2         2        3    ...            3         1      3        2        1   
3         2        3    ...            4         2      3        3        1   
4         3        3    ...            4         4      3        3        2   

   job  depends  te



Table of metrics for 10 Folds CV of model with MAX_TREE_DEPTH = 10

           Precision  Recall/TPR  F1_Score  Accuracy
1st Fold    0.770833    0.755102  0.767677  0.676056
2nd Fold    0.736842    0.857143  0.792453  0.690141
3rd Fold    0.750000    0.673469  0.715789  0.647887
4th Fold    0.760000    0.812500  0.775510  0.685714
5th Fold    0.735849    0.812500  0.772277  0.671429
6th Fold    0.807692    0.875000  0.840000  0.771429
7th Fold    0.791667    0.812500  0.804124  0.728571
8th Fold    0.773585    0.854167  0.811881  0.724638
9th Fold    0.781818    0.895833  0.846154  0.753623
10th Fold   0.750000    0.791667  0.780000  0.652174

Avg_Precision = 0.7658286557037799
Avg_Recall = 0.8139880952380952
Avg_F1 = 0.7905865248966926
Avg_Accuracy = 0.7001662146793806

Displaying Feature Importance for the best Model:

     Feature  Importance
0        age    0.166505
1     amount    0.158877
2   duration    0.115608
3    depends    0.003254
4    foreign    0.000000
5    purpose   